In [ ]:
import utils.dataloader
import pandas as pd
import numpy as np

Cs = np.logspace(-1,2,5)
pca_red = np.logspace(1.38,np.log10(64**3)/np.log10(10),20).astype(int)
df = pd.DataFrame(index = pca_red,columns=Cs)

df.loc[23,0.1] = 3

print(df)


In [4]:
from sklearn.svm import SVC
import numpy as np
from sklearn import decomposition
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
import os
import h5py
import pandas as pd
import multiprocessing as mp

data_path = os.path.join('.','tests','test_output','data.hdf5')

dat = h5py.File(data_path, 'r')

X = np.array(dat.get('data'))
Y = np.array(dat.get('data_label'))
dat.close()
Cs = np.logspace(-1,2,5)
pca_red = np.logspace(1.38,np.log10(64**3)/np.log10(10),10).astype(int)

mean_lin_df = pd.DataFrame(index = pca_red,columns=Cs)
var_lin_df = pd.DataFrame(index = pca_red,columns=Cs)


#for c in Cs:
#    for comp in pca_red:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

linear_svc = SVC(C = 10, kernel = 'linear',gamma = 'auto')
linear_svc.fit(X_train,y_train)

score_lin = linear_svc.score(X_test,y_test)

print(score_lin)
        


# Number of features to take 

featurenum_list = [10, 100, 1000, 10000, 100000] 

# Transform the data using PCA 

from sklearn.decomposition import PCA
pca_list=[]
for x in featurenum_list: 
    pca = PCA(n_components=x)
    X_pca = pca.fit_transform(X)
    pca_list.append(X_pca)

# Transform the data using LDA, not for loop needed, only take feature numbers is #(labels - 1)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda_list=[] 
lda = LDA()
lda.fit(X,Y)
X_lda = lda.transform(X)
lda_list.append(X_lda)

#Select the best features 
from sklearn.feature_selection import SelectKBest, chi2
# Try for different k 
selkb_list = [] 
for x in featurenum_list:
    X_selkbest = SelectKBest(chi2, k=x).fit_transform(X, Y)
    selkb_list.append(X_selkbest)
    
######################## Decision Tree 
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.datasets import load_iris
from sklearn import tree

for x in selkb_list: # or pca_list
    Decacc = [] 
    rkfDec = RepeatedKFold(n_splits=10, n_repeats=10, random_state=None)
    for train_index, test_index in rkfDec.split(x):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = Y[train_index], Y[test_index] 
        clfDec = tree.DecisionTreeClassifier()
        clfDec = clf.fit(X_train, y_train)
        Decacc.append(clfDec.score(X_test,y_test)*100)    
    print("For SELKB n =",x,"mean =",np.mean(Decacc),"var =",np.var(Decacc))  
    
for x in pca_list: # or pca_list
    Decacc = [] 
    rkfDec = RepeatedKFold(n_splits=10, n_repeats=10, random_state=None)
    for train_index, test_index in rkfDec.split(x):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = Y[train_index], Y[test_index] 
        clfDec = tree.DecisionTreeClassifier()
        clfDec = clf.fit(X_train, y_train)
        Decacc.append(clfDec.score(X_test,y_test)*100)    
    print("For PCA, n =",x,"mean =",np.mean(Decacc),"var =",np.var(Decacc))

####################### KNN 
from sklearn.neighbors import KNeighborsClassifier
K = [1, 3, 5, 11]
for x in selkb_list: # 
    for k in K: 
        KNNacc = [] 
        rkfKNN = RepeatedKFold(n_splits=10, n_repeats=10, random_state=None)
        for train_index, test_index in rkfKNN.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index] 
            clfKNN = KNeighborsClassifier(n_neighbors=k)
            clfKNN = clfKNN.fit(X_train, y_train)
            KNNacc.append(clfKNN.score(X_test,y_test)*100)
        print("For SELKB n =",x,"k=",k,"mean =",np.mean(KNNacc),"var =",np.var(KNNacc))  

for x in pca_list: # or pca_list
    for k in K: 
        KNNacc = [] 
        rkfKNN = RepeatedKFold(n_splits=10, n_repeats=10, random_state=None)
        for train_index, test_index in rkfKNN.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index] 
            clfKNN = KNeighborsClassifier(n_neighbors=k)
            clfKNN = clfKNN.fit(X_train, y_train)
            KNNacc.append(clfKNN.score(X_test,y_test)*100)
        print("For PCA n =",x,"k=",k,"mean =",np.mean(KNNacc),"var =",np.var(KNNacc))  


OSError: Unable to open file (unable to open file: name = '.\tests\test_output\data.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import multiprocessing as mp

print(mp.cpu_count())